In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from tqdm import tqdm
from math import log10
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV, KFold

In [0]:
df = pd.read_csv('dba.csv')

In [93]:
df.head()

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [94]:
df = shuffle(df).reset_index(drop=True)
train_data = df[472:]
test_data = df[:472]
print(train_data.shape, test_data.shape)

(900, 5) (472, 5)


In [0]:
def get_c_range(x_train, y_train):
    c_ = np.logspace(-5, 8, 10)
    scores = []
    for c in c_:
        svc = LinearSVC(penalty='l1', C=c, dual=False)
        svc.fit(x_train, y_train)
        scores.append(svc.score(x_train, y_train))
    scores = np.array(scores)
    ind = np.argwhere(scores > 0.9).flatten()
    c_1 = c_[ind[0]]
    c_2 = c_[ind[-1]]
    return c_1, c_2

In [0]:
def linearSVC(x_train, y_train):
    c_l, c_h = get_c_range(x_train, y_train)
    parameters = {'C':np.logspace(log10(c_l), log10(c_h), 20)}

    svc = LinearSVC(penalty='l1', dual=False)
    n_splits = 10 if (len(x_train) > 10) else 5

    kf = StratifiedKFold(n_splits=n_splits, shuffle=True)
    clf = GridSearchCV(svc, parameters, cv=kf, scoring='accuracy', return_train_score=True)
    clf.fit(x_train, y_train)
    return clf.best_estimator_

In [0]:
results = []
test_data = shuffle(test_data)
x_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]

for i in tqdm(range(50)):
    each = []
    
    remaining, pool = train_test_split(train_data, test_size=10)
    while(len(pool['class'].unique()) == 1 or pool['class'].value_counts().min() < 2):
        remaining, pool = train_test_split(train_data, test_size=10)
    clf = linearSVC(pool.iloc[:, :-1], pool['class'])
    each.append(1 - clf.score(x_test, y_test))
    
    for j in range(88):
        remaining, selected = train_test_split(remaining, test_size=10)
        pool = pool.append(selected)
        clf = linearSVC(pool.iloc[:, :-1], pool['class'])
        each.append(1 - clf.score(x_test, y_test))
        
    clf = linearSVC(train_data.iloc[:, :-1], train_data['class'])
    each.append(1 - clf.score(x_test, y_test))
    
    results.append(np.mean(each))
pd.DataFrame(results).to_csv('Error.csv', index=False)